##DEEP LEARNING
##Lab 3

In [21]:
import torch
import torchvision
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import transforms, datasets
from torch import optim
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
import pandas as pd
from sklearn.manifold import TSNE
from torchsummary import summary
from sklearn.metrics import confusion_matrix
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr

In [2]:
# Prepare CIFAR-10 dataset
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
])

train_data = datasets.CIFAR10(root='~/torch_datasets', train=True, download=True, transform=transform)
test_data = datasets.CIFAR10(root='~/torch_datasets', train=False, download=True, transform=transform)


Files already downloaded and verified
Files already downloaded and verified


In [3]:
# Define hyperparameters
random_seed = 10
torch.manual_seed(random_seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True
batch_size = 32
epochs = 10
learning_rate = 1e-3

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

In [25]:
# Initialize the autoencoder
autoencoder = SimpleAutoencoder().to(device)

# Print the parameters of the autoencoder
print("Autoencoder Parameters:")
for name, param in autoencoder.named_parameters():
    print(f"Parameter name: {name}, Shape: {param.shape}")

# Optionally, print the total number of parameters
total_params = sum(p.numel() for p in autoencoder.parameters())
print(f"Total number of parameters: {total_params}")


Autoencoder Parameters:
Parameter name: encoder.0.weight, Shape: torch.Size([16, 3, 3, 3])
Parameter name: encoder.0.bias, Shape: torch.Size([16])
Parameter name: encoder.3.weight, Shape: torch.Size([8, 16, 3, 3])
Parameter name: encoder.3.bias, Shape: torch.Size([8])
Parameter name: decoder.0.weight, Shape: torch.Size([8, 16, 2, 2])
Parameter name: decoder.0.bias, Shape: torch.Size([16])
Parameter name: decoder.2.weight, Shape: torch.Size([16, 3, 2, 2])
Parameter name: decoder.2.bias, Shape: torch.Size([3])
Total number of parameters: 2331


In [4]:
# Define the autoencoder architecture
class SimpleAutoencoder(nn.Module):
    def __init__(self):
        super(SimpleAutoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1),
            nn.LeakyReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(16, 8, kernel_size=3, stride=1, padding=1),
            nn.LeakyReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(8, 16, kernel_size=2, stride=2),
            nn.LeakyReLU(inplace=True),
            nn.ConvTranspose2d(16, 3, kernel_size=2, stride=2),
            nn.Tanh()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [5]:
# Initialize the autoencoder
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
autoencoder = SimpleAutoencoder().to(device)

# Loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(autoencoder.parameters(), lr=learning_rate)

# Pretrain the autoencoder for the first convolutional block
pretrain_losses_block1 = []
for epoch in range(epochs):
    running_loss = 0.0
    for data in train_loader:
        inputs, _ = data
        inputs = inputs.to(device)

        optimizer.zero_grad()
        outputs = autoencoder(inputs)
        loss = criterion(outputs, inputs)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    epoch_loss = running_loss / len(train_loader)
    pretrain_losses_block1.append(epoch_loss)
    print(f"Block 1 Pretraining: Epoch [{epoch + 1}/{epochs}], Loss: {epoch_loss:.4f}")

Block 1 Pretraining: Epoch [1/10], Loss: 0.0201
Block 1 Pretraining: Epoch [2/10], Loss: 0.0075
Block 1 Pretraining: Epoch [3/10], Loss: 0.0063
Block 1 Pretraining: Epoch [4/10], Loss: 0.0058
Block 1 Pretraining: Epoch [5/10], Loss: 0.0056
Block 1 Pretraining: Epoch [6/10], Loss: 0.0054
Block 1 Pretraining: Epoch [7/10], Loss: 0.0053
Block 1 Pretraining: Epoch [8/10], Loss: 0.0052
Block 1 Pretraining: Epoch [9/10], Loss: 0.0051
Block 1 Pretraining: Epoch [10/10], Loss: 0.0050


In [6]:
# Pretrain the autoencoder for the second convolutional block
autoencoder.decoder[0] = nn.ConvTranspose2d(8, 16, kernel_size=2, stride=2)
optimizer = optim.Adam(autoencoder.parameters(), lr=learning_rate)

pretrain_losses_block2 = []
for epoch in range(epochs):
    running_loss = 0.0
    for data in train_loader:
        inputs, _ = data
        inputs = inputs.to(device)

        optimizer.zero_grad()
        outputs = autoencoder(inputs)
        loss = criterion(outputs, inputs)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    epoch_loss = running_loss / len(train_loader)
    pretrain_losses_block2.append(epoch_loss)
    print(f"Block 2 Pretraining: Epoch [{epoch + 1}/{epochs}], Loss: {epoch_loss:.4f}")

Block 2 Pretraining: Epoch [1/10], Loss: 0.0085
Block 2 Pretraining: Epoch [2/10], Loss: 0.0053
Block 2 Pretraining: Epoch [3/10], Loss: 0.0051
Block 2 Pretraining: Epoch [4/10], Loss: 0.0050
Block 2 Pretraining: Epoch [5/10], Loss: 0.0049
Block 2 Pretraining: Epoch [6/10], Loss: 0.0049
Block 2 Pretraining: Epoch [7/10], Loss: 0.0048
Block 2 Pretraining: Epoch [8/10], Loss: 0.0048
Block 2 Pretraining: Epoch [9/10], Loss: 0.0047
Block 2 Pretraining: Epoch [10/10], Loss: 0.0047


In [7]:
# Evaluate performance on the test set
autoencoder.eval()
test_losses = []
with torch.no_grad():
    for data in test_loader:
        inputs, _ = data
        inputs = inputs.to(device)

        outputs = autoencoder(inputs)
        loss = criterion(outputs, inputs)
        test_losses.append(loss.item())

# Calculate average test loss
avg_test_loss = sum(test_losses) / len(test_losses)
print(f"Average Test Loss: {avg_test_loss:.4f}")

Average Test Loss: 0.0047


In [10]:
# Create pretraining loss curves
fig = go.Figure()

fig.add_trace(go.Scatter(x=list(range(epochs)), y=pretrain_losses_block1, mode='lines', name='Block 1 Pretraining'))

fig.add_trace(go.Scatter(x=list(range(epochs)), y=pretrain_losses_block2, mode='lines', name='Block 2 Pretraining'))

# Update layout
fig.update_layout(title='Pretraining Loss Curves',
                  xaxis_title='Epochs',
                  yaxis_title='Loss',
                  legend=dict(x=0.7, y=0.9),
                  template='plotly_white')

fig.show()

In [17]:
# Visualize some input images and their reconstructions using Plotly
num_images_to_show = 5

# Plotting histograms of reconstruction errors
fig_hist = px.histogram(x=test_losses, nbins=50, title='Histogram of Reconstruction Errors')
fig_hist.update_layout(xaxis_title='Reconstruction Error', yaxis_title='Frequency')

fig_hist.show()

In [22]:
# Get encoded features for the entire test dataset
encoded_features = []
with torch.no_grad():
    for data in test_loader:
        inputs, _ = data
        inputs = inputs.to(device)
        encoded_output = autoencoder.encoder(inputs).cpu().numpy()
        encoded_features.append(encoded_output)

encoded_features = np.concatenate(encoded_features, axis=0)

# Flatten the encoded features
encoded_features_flattened = encoded_features.reshape(encoded_features.shape[0], -1)

# Reduce dimensionality using T-SNE
tsne = TSNE(n_components=2, random_state=42)
tsne_output = tsne.fit_transform(encoded_features_flattened)

# Plot T-SNE visualization
tsne_df = pd.DataFrame(tsne_output, columns=['TSNE component 1', 'TSNE component 2'])
tsne_df['Label'] = test_data.targets

fig = px.scatter(tsne_df, x='TSNE component 1', y='TSNE component 2', color='Label',
                 title='T-SNE Visualization')
fig.show()


In [23]:
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr

# Evaluate SSIM and PSNR
ssim_scores = []
psnr_scores = []

with torch.no_grad():
    for data in test_loader:
        inputs, _ = data
        inputs = inputs.to(device)

        outputs = autoencoder(inputs).cpu().numpy()
        inputs = inputs.cpu().numpy()

        for i in range(len(inputs)):
            ssim_score = ssim(inputs[i].transpose((1, 2, 0)), outputs[i].transpose((1, 2, 0)), multichannel=True)
            psnr_score = psnr(inputs[i].transpose((1, 2, 0)), outputs[i].transpose((1, 2, 0)))

            ssim_scores.append(ssim_score)
            psnr_scores.append(psnr_score)

avg_ssim = np.mean(ssim_scores)
avg_psnr = np.mean(psnr_scores)
print(f"Average SSIM: {avg_ssim:.4f}")
print(f"Average PSNR: {avg_psnr:.4f}")


<ipython-input-23-6d39dcf63c4f>:17: FutureWarning:

`multichannel` is a deprecated argument name for `structural_similarity`. It will be removed in version 1.0. Please use `channel_axis` instead.



Average SSIM: 0.8437
Average PSNR: 23.8576


For the evaluation an autoencoder's performance in reconstructing images can be done by:

Reconstruction Loss

Average Test Loss

Reconstruction Error Histogram

Peak Signal-to-Noise Ratio (PSNR)

Structural Similarity Index (SSIM)

T-SNE Plot